In [1]:
import numpy as np 
import pandas as pd 

## Loading Data

In [ ]:
from datasets import load_dataset
import pandas as pd
# dataset = pd.read_csv("/kaggle/input/doc-query/doc_query_pairs.train.tsv", sep='\t')
df = pd.read_csv("doc_query_pairs.train.tsv", sep='\t')
df.columns = ['Text', 'Query']

In [3]:
df.head()

,Text,Query
0,The approach is based on a theory of justice t...,_________ justice is designed to repair the ha...
1,"Colorâurine can be a variety of colors, most...",what color is amber urine
2,Inborn errors of bile acid synthesis can produ...,is autoimmune hepatitis a bile acid synthesis ...
3,The word convict here (elegcw /elegxo) means t...,elegxo meaning
4,In-home tutors can earn anywhere from $10 to $...,how much does an average person make for tutoring


In [4]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:

def preprocess_function(examples):
    
    input_texts = ["generate response: " + text + " </s> " + query for text, query in zip(examples["Text"], examples["Query"])]
    
    # Tokenize the combined input text
    model_inputs = tokenizer(
        input_texts,
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    
    # Since T5 can use the same tokenizer for inputs and labels, we also tokenize the 'Query' as labels
    labels = tokenizer(
        examples["Query"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )


    labels["input_ids"] = [
        (label if label != tokenizer.pad_token_id else -100)
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:

# Split the dataset into training and test sets
train_test_split = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

## Model Training

In [7]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, T5Tokenizer
import torch

# Load the model with a specified cache directory
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Check if a GPU is available and set the device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)  # Ensure the model is on the correct device
print(f"Model is using device: {device}")


2024-12-10 22:54:21.292572: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-10 22:54:21.317843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-10 22:54:21.834556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Model is using device: cuda


In [8]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=150,  # Evaluation and saving happens every 500 steps
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=3,  # Only last 3 models are saved
    num_train_epochs=2,
    report_to="none",  # Or 'wandb' for Weights & Biases 
    fp16=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

/home/nirmal/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/nirmal/miniconda3/envs/tf/lib/python3.9/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [9]:
# Start training
trainer.train()


Step,Training Loss,Validation Loss
150,No log,0.001787
300,No log,0.000603
450,No log,0.000517


TrainOutput(global_step=450, training_loss=0.6209653049045138, metrics={'train_runtime': 353.2162, 'train_samples_per_second': 10.192, 'train_steps_per_second': 1.274, 'total_flos': 2192248406016000.0, 'train_loss': 0.6209653049045138, 'epoch': 2.0})